# Particle Size Distribution

**Note:** Unfortunately, GitHub does not render Plotly charts. To render the interactive charts in this notebook, you will have to paste the notebook URL on [nbviewer](https://nbviewer.org).

In [1]:
# configuration for display on nbviewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
import math

import pandas as pd
import plotly.graph_objects as go

In [3]:
input_data = {
    "sieve_size": [4, 2, 1, 0.5, 0.25, 0.125, 0.025],
    "soil_retained_%": [0.5, 2, 40, 50, 75, 25, 7.5]
}
df = pd.DataFrame(input_data)

df

,sieve_size,soil_retained_%
0,4.000,0.5
1,2.000,2.0
2,1.000,40.0
3,0.500,50.0
4,0.250,75.0
5,0.125,25.0
6,0.025,7.5


In [4]:
def log(value):
    return -math.log(value, 2)

def assign_wentworth_size_class(phi_scale):
    size_mapping = {
        (-10, -8): "Boulder",
        (-8, -6): "Cobble",
        (-6, -2): "Pebble",
        (-2, -1): "Granule",
        (-1, 0): "Very Coarse Sand",
        (0, 1): "Coarse Sand",
        (1, 2): "Medium Sand",
        (2, 3): "Fine Sand",
        (3, 4): "Very Fine Sand",
        (4, 5): "Coarse Silt",
        (5, 6): "Medium Silt",
        (6, 7): "Fine Silt",
        (7, 8): "Very Fine Silt",
        (8, float('inf')): "Clay"
    }
    
    for interval, size_class in size_mapping.items():
        if interval[0] < phi_scale <= interval[1]:
            return size_class


In [5]:

df["phi_scale"] = df["sieve_size"].apply(log).round(decimals=2)
df

,sieve_size,soil_retained_%,phi_scale
0,4.000,0.5,-2.00
1,2.000,2.0,-1.00
2,1.000,40.0,-0.00
3,0.500,50.0,1.00
4,0.250,75.0,2.00
5,0.125,25.0,3.00
6,0.025,7.5,5.32


In [6]:
# handle negative zeros
df[df == -0] = 0

# assign wentworth size class and re-order the columns
df["wentworth_size_class"] = df["phi_scale"].apply(assign_wentworth_size_class)
df = df.reindex(columns=["sieve_size", "phi_scale", "wentworth_size_class", "soil_retained_%"])

df

,sieve_size,phi_scale,wentworth_size_class,soil_retained_%
0,4.000,-2.00,Pebble,0.5
1,2.000,-1.00,Granule,2.0
2,1.000,0.00,Very Coarse Sand,40.0
3,0.500,1.00,Coarse Sand,50.0
4,0.250,2.00,Medium Sand,75.0
5,0.125,3.00,Fine Sand,25.0
6,0.025,5.32,Medium Silt,7.5


In [7]:
total_soil_retained = df["soil_retained_%"].sum()

df["percent_retained_%"] = (df["soil_retained_%"] / total_soil_retained) * 100
df["cumulative_retained_%"] = df["percent_retained_%"].cumsum()
df["percent_passing_%"] = 100 - df["cumulative_retained_%"]

df

,sieve_size,phi_scale,wentworth_size_class,soil_retained_%,percent_retained_%,cumulative_retained_%,percent_passing_%
0,4.000,-2.00,Pebble,0.5,0.25,0.25,99.75
1,2.000,-1.00,Granule,2.0,1.00,1.25,98.75
2,1.000,0.00,Very Coarse Sand,40.0,20.00,21.25,78.75
3,0.500,1.00,Coarse Sand,50.0,25.00,46.25,53.75
4,0.250,2.00,Medium Sand,75.0,37.50,83.75,16.25
5,0.125,3.00,Fine Sand,25.0,12.50,96.25,3.75
6,0.025,5.32,Medium Silt,7.5,3.75,100.00,0.00


In [8]:
layout = go.Layout(
    title="<b>Particle Size Distribution</b>",
    xaxis=dict(
        title="Particle Size, mm (log scale)",
        type="log",
        autorange="reversed"
    ),
    yaxis=dict(title="<b>% of Finer</b>"),
    font_size=14,
    width=900,
    height=580
)

trace = go.Scatter(
    x=df["sieve_size"],
    y=df["percent_passing_%"],
    customdata=df["wentworth_size_class"],
    hovertemplate="<br>".join(["<b>%{customdata}</b><br>", "<b>Grain size:</b> %{x}", "<b>% of Finer:</b> %{y}"]),
    line=dict(width=2.5, shape="spline"),
    marker=dict(size=10, symbol="circle"),
    name=""
)

go.Figure(layout=layout, data=[trace])